In [1]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import optuna
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

from src.utils import get_kfold_data, convert_non_numeric_to_numeric, calculate_r2_score, calculate_metrics
from src.normalisation import Normaliser
from src.constants import *


c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv(DATA_PATH)

In [3]:
data.head()

,outcome,carat,cut,color,clarity,depth,table,price,x,y,...,a6,a7,a8,a9,a10,b6,b7,b8,b9,b10
0,-26.701232,1.14,Ideal,G,VS1,62.3,56.0,7948,6.73,6.70,...,0.168836,-0.273758,1.107832,1.247795,0.482344,0.489511,-0.321138,0.573382,0.446871,-1.990581
1,6.548093,0.38,Premium,H,VS2,60.5,59.0,898,4.69,4.66,...,-0.256549,0.315373,-0.030326,-0.114335,-1.059588,-1.761360,-1.343951,-1.002550,-0.225030,-0.446653
2,6.612562,0.50,Very Good,E,SI1,60.7,58.0,1351,5.09,5.13,...,-1.193327,-0.657307,-0.591726,-0.446856,-0.765286,-0.816544,-1.397794,-0.477130,0.810509,1.725131
3,-5.073562,0.70,Premium,D,SI1,61.2,58.0,2512,5.74,5.70,...,-1.740788,-1.778860,-0.825070,0.444932,1.173109,0.453606,-0.263440,0.246210,-0.850503,-0.412950
4,-14.436557,0.83,Ideal,G,SI2,62.4,54.0,2751,6.01,6.08,...,-0.859322,1.409268,0.861992,1.109063,-1.436722,-1.461618,0.081787,0.258087,0.851146,2.204813


Inspecting columns

In [4]:
# Find columns
all_columns = data.columns.tolist()
print(all_columns)

numeric_columns = data.select_dtypes(include=["number"]).columns.tolist()
numeric_columns.remove("outcome") # Remove the target column
print(numeric_columns)

non_numeric_columns = data.select_dtypes(exclude=["number"]).columns.tolist()
print(non_numeric_columns)

['outcome', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['cut', 'color', 'clarity']


In [5]:
for non_numeric_column in non_numeric_columns:
    print(data[non_numeric_column].value_counts())

cut
Ideal        4040
Premium      2439
Very Good    2296
Good          925
Fair          300
Name: count, dtype: int64
color
G    2120
E    1873
F    1746
H    1506
D    1246
I     983
J     526
Name: count, dtype: int64
clarity
SI1     2408
VS2     2256
SI2     1743
VS1     1503
VVS2     951
VVS1     675
IF       318
I1       146
Name: count, dtype: int64


Converting non-numeric features to numerical features

In [6]:
data = convert_non_numeric_to_numeric(data=data)
print(data)

['G', 'E', 'F', 'H', 'D', 'I', 'J']
        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...      

Normalise data using each columns respective mean and std.

In [7]:
print(data)

        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...        b8        b9       b10  colour_G  

In [8]:
normaliser = Normaliser()
for column in numeric_columns:
    print(data[column])
    data[column] = normaliser.standardise(data[column])
    print("after", data[column])
    break

0       1.14
1       0.38
2       0.50
3       0.70
4       0.83
        ... 
9995    0.33
9996    1.01
9997    0.52
9998    0.31
9999    0.37
Name: carat, Length: 10000, dtype: float64
after 0       0.723643
1      -0.886369
2      -0.632156
3      -0.208469
4       0.066928
          ...   
9995   -0.992290
9996    0.448246
9997   -0.589788
9998   -1.034659
9999   -0.907553
Name: carat, Length: 10000, dtype: float64


In [9]:
print(data)

        outcome     carat  cut  clarity  depth  table  price     x     y  \
0    -26.701232  0.723643    0        3   62.3   56.0   7948  6.73  6.70   
1      6.548093 -0.886369    1        4   60.5   59.0    898  4.69  4.66   
2      6.612562 -0.632156    2        5   60.7   58.0   1351  5.09  5.13   
3     -5.073562 -0.208469    1        5   61.2   58.0   2512  5.74  5.70   
4    -14.436557  0.066928    0        6   62.4   54.0   2751  6.01  6.08   
...         ...       ...  ...      ...    ...    ...    ...   ...   ...   
9995  10.718277 -0.992290    0        3   62.6   57.0   1002  4.42  4.40   
9996 -12.246698  0.448246    4        5   69.5   55.0   4853  6.00  5.94   
9997  11.122516 -0.589788    2        6   57.9   61.0   1273  5.28  5.33   
9998 -24.730782 -1.034659    0        0   62.0   54.0    801  4.35  4.39   
9999   8.735755 -0.907553    2        5   59.9   59.0    649  4.68  4.70   

         z  ...        b8        b9       b10  colour_G  colour_E  colour_F  \
0     4.

In [10]:
kfold_data = get_kfold_data(data=data, k=NUM_FOLDS, reproducibility_seed=REPRODUCIBILITY_SEED)

Fold: 0/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 1/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 2/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 3/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 4/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%



Define models and hyperparameter tuning objectives for each model

In [11]:
def objective(model_type, trial, x_train, y_train, x_val, y_val):
    if model_type == lgb.LGBMRegressor:
        parameters = {
                    "objective": "regression",
                    "metric": "rmse",
                    "n_estimators": 100,
                    "verbosity": -1,
                    "bagging_freq": 1,
                    "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
                    "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
                    "subsample": trial.suggest_float("subsample", 0.05, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
                    "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
                    "seed": REPRODUCIBILITY_SEED
        }
    elif model_type == xgb.XGBRegressor:
        parameters = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "n_estimators": 100,
            "eta": trial.suggest_float("eta", 1e-2, 0.2, log=True),
            "gamma": trial.suggest_float("gamma", 1e-8, 10, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 6),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "seed": REPRODUCIBILITY_SEED
        }
    elif model_type == RandomForestRegressor:
        parameters = {
            "n_estimators": 100,
            "criterion": trial.suggest_categorical("criterion", ["absolute_error", "squared_error"]),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "bootstrap": True,
            "oob_score": False,
            "n_jobs": -1,
            "random_state": REPRODUCIBILITY_SEED
        }

    model = model_type(**parameters) # Create the model
    model.fit(x_train, y_train)
    predictions = model.predict(x_val)
    metrics = calculate_metrics(targets=y_val, preds=predictions)
    rmse = metrics["rmse"]
    return rmse


models = {
        # "linear_regression": LinearRegression(),
        # "xgb": xgb.XGBRegressor,
        "random_forest": RandomForestRegressor,
        # "gradient_boosting": GradientBoostingRegressor(**HYPERPARAMETERS_2),
        # "ada_boost": AdaBoostRegressor(),
        # "ridge": Ridge(),
        # "lasso": Lasso(),
        #"lgbm": lgb.LGBMRegressor
        }

In [12]:
# Train + Validate models
metrics = ["mae", "mse", "rmse", "pcc", "spearman_r", "r2_score"]
model_scores = {model_name: defaultdict(list) for model_name in models.keys()}

for fold in range(NUM_FOLDS):
    fold_data = kfold_data[fold]
     
    # Extract data
    train_data = fold_data["train"]
    val_data = fold_data["val"]
    test_data = kfold_data[fold]["test"]

    train_y = train_data["outcome"]
    val_y = val_data["outcome"]
    test_y = test_data["outcome"]

    train_x = train_data.drop(columns=["outcome"])
    val_x = val_data.drop(columns=["outcome"])
    test_x = test_data.drop(columns=["outcome"])

    # print(f"Fold {fold+1}/{NUM_FOLDS}")
    # print(f"Train data shape: {train_x.shape} | Train target shape: {train_y.shape}")
    # print(f"Val data shape: {val_x.shape} | Val target shape: {val_y.shape}")
    # print(f"Test data shape: {test_x.shape} | Test target shape: {test_y.shape}")

    # Train model
    for model_name, model in models.items():
        study = optuna.create_study(direction="minimize")
        study.optimize(lambda trial: objective(trial=trial, 
                                               model_type=model, 
                                               x_train=train_x, 
                                               y_train=train_y, 
                                               x_val=val_x, 
                                               y_val=val_y
                                               ), n_trials=30)
        
        # Train model with best hyperparameters
        best_fold_params = study.best_params
        model = model(**best_fold_params)
        model.fit(train_x, train_y)
        preds = model.predict(val_x)

        metrics = calculate_metrics(targets=val_y, preds=preds)
        mae = metrics["mae"]
        mse = metrics["mse"]
        rmse = metrics["rmse"]
        pcc = metrics["pcc"]
        spearman_r = metrics["spearman_r"]
        r2_score = metrics["r2_score"]

        for metric in metrics:
            model_scores[model_name][metric].append(metrics[metric])

        print(f"Fold: {fold+1}/{NUM_FOLDS}")
        print(f"Model name: {model_name}")
        print(f"MAE: {mae}")
        print(f"MSE: {mse}")
        print(f"RMSE: {rmse}")
        print(f"PCC: {pcc}")
        print(f"Spearman R: {spearman_r}")
        print(f"R2 Score: {r2_score}")
        print()

# Compute average scores
for model_name, model_metrics in model_scores.items():
    for metric, scores in model_metrics.items():
        model_scores[model_name][metric] = sum(scores) / len(scores)
    model_scores[model_name] = dict(model_scores[model_name])
print(model_scores)

[I 2025-02-16 14:50:42,639] A new study created in memory with name: no-name-e0d4bcb4-1141-48c2-9226-9abd3016d42e
[I 2025-02-16 14:50:42,793] Trial 0 finished with value: 9.977295456821766 and parameters: {'criterion': 'squared_error', 'max_features': 'log2', 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 8}. Best is trial 0 with value: 9.977295456821766.
[I 2025-02-16 14:50:42,911] Trial 1 finished with value: 10.523798093269397 and parameters: {'criterion': 'squared_error', 'max_features': 'sqrt', 'max_depth': 4, 'min_samples_split': 8, 'min_samples_leaf': 6}. Best is trial 0 with value: 9.977295456821766.
[I 2025-02-16 14:50:43,030] Trial 2 finished with value: 10.329813117735169 and parameters: {'criterion': 'squared_error', 'max_features': 'sqrt', 'max_depth': 5, 'min_samples_split': 8, 'min_samples_leaf': 4}. Best is trial 0 with value: 9.977295456821766.
[I 2025-02-16 14:50:44,728] Trial 3 finished with value: 10.192338678089738 and parameters: {'criterion': 'absol

Fold: 1/5
Model name: random_forest
MAE: 7.86069739764213
MSE: 97.49721962062594
RMSE: 9.874068038079642
PCC: 0.6707914550054646
Spearman R: 0.6769090026988293
R2 Score: 0.42540594204775406



[I 2025-02-16 14:50:57,745] Trial 1 finished with value: 11.099838843450364 and parameters: {'criterion': 'squared_error', 'max_features': 'log2', 'max_depth': 3, 'min_samples_split': 7, 'min_samples_leaf': 8}. Best is trial 0 with value: 9.684684760377667.
[I 2025-02-16 14:50:57,848] Trial 2 finished with value: 10.689828823244252 and parameters: {'criterion': 'squared_error', 'max_features': 'sqrt', 'max_depth': 3, 'min_samples_split': 9, 'min_samples_leaf': 9}. Best is trial 0 with value: 9.684684760377667.
[I 2025-02-16 14:50:57,989] Trial 3 finished with value: 9.805328345229404 and parameters: {'criterion': 'squared_error', 'max_features': 'log2', 'max_depth': 9, 'min_samples_split': 9, 'min_samples_leaf': 6}. Best is trial 0 with value: 9.684684760377667.
[I 2025-02-16 14:50:59,534] Trial 4 finished with value: 10.658236324518688 and parameters: {'criterion': 'absolute_error', 'max_features': 'sqrt', 'max_depth': 3, 'min_samples_split': 3, 'min_samples_leaf': 1}. Best is trial 0

Fold: 2/5
Model name: random_forest
MAE: 7.644699233051203
MSE: 92.39509206315807
RMSE: 9.61223657964982
PCC: 0.6640184569016031
Spearman R: 0.6629222237977437
R2 Score: 0.42088440294468554



[I 2025-02-16 14:51:14,210] Trial 1 finished with value: 10.312023927915442 and parameters: {'criterion': 'absolute_error', 'max_features': 'sqrt', 'max_depth': 4, 'min_samples_split': 10, 'min_samples_leaf': 10}. Best is trial 0 with value: 9.653522372656239.
[I 2025-02-16 14:51:15,822] Trial 2 finished with value: 10.484047387856556 and parameters: {'criterion': 'absolute_error', 'max_features': 'log2', 'max_depth': 5, 'min_samples_split': 4, 'min_samples_leaf': 7}. Best is trial 0 with value: 9.653522372656239.
[I 2025-02-16 14:51:17,556] Trial 3 finished with value: 9.941281832398305 and parameters: {'criterion': 'absolute_error', 'max_features': 'log2', 'max_depth': 8, 'min_samples_split': 3, 'min_samples_leaf': 10}. Best is trial 0 with value: 9.653522372656239.
[I 2025-02-16 14:51:17,670] Trial 4 finished with value: 10.27423255479268 and parameters: {'criterion': 'squared_error', 'max_features': 'sqrt', 'max_depth': 4, 'min_samples_split': 6, 'min_samples_leaf': 6}. Best is tri

Fold: 3/5
Model name: random_forest
MAE: 7.581115573554627
MSE: 90.85210611017075
RMSE: 9.531637115950794
PCC: 0.6839614342936976
Spearman R: 0.6911566586549449
R2 Score: 0.44296150939682877



[I 2025-02-16 14:51:29,688] Trial 1 finished with value: 10.089140746334486 and parameters: {'criterion': 'absolute_error', 'max_features': 'log2', 'max_depth': 6, 'min_samples_split': 3, 'min_samples_leaf': 9}. Best is trial 0 with value: 9.46296548162563.
[I 2025-02-16 14:51:29,816] Trial 2 finished with value: 9.81082306993021 and parameters: {'criterion': 'squared_error', 'max_features': 'log2', 'max_depth': 8, 'min_samples_split': 8, 'min_samples_leaf': 4}. Best is trial 0 with value: 9.46296548162563.
[I 2025-02-16 14:51:29,951] Trial 3 finished with value: 9.687527755667887 and parameters: {'criterion': 'squared_error', 'max_features': 'log2', 'max_depth': 9, 'min_samples_split': 4, 'min_samples_leaf': 5}. Best is trial 0 with value: 9.46296548162563.
[I 2025-02-16 14:51:30,064] Trial 4 finished with value: 10.653645424925816 and parameters: {'criterion': 'squared_error', 'max_features': 'log2', 'max_depth': 4, 'min_samples_split': 5, 'min_samples_leaf': 8}. Best is trial 0 with

Fold: 4/5
Model name: random_forest
MAE: 7.470128195856332
MSE: 89.34482706836675
RMSE: 9.452239262120207
PCC: 0.6820414673380248
Spearman R: 0.6893622155711779
R2 Score: 0.44441302926206183



[I 2025-02-16 14:51:48,884] Trial 1 finished with value: 10.693555499143352 and parameters: {'criterion': 'squared_error', 'max_features': 'sqrt', 'max_depth': 3, 'min_samples_split': 9, 'min_samples_leaf': 8}. Best is trial 0 with value: 9.635666261034775.
[I 2025-02-16 14:51:49,027] Trial 2 finished with value: 9.579209481161326 and parameters: {'criterion': 'squared_error', 'max_features': 'sqrt', 'max_depth': 9, 'min_samples_split': 9, 'min_samples_leaf': 1}. Best is trial 2 with value: 9.579209481161326.
[I 2025-02-16 14:51:49,154] Trial 3 finished with value: 9.727364090995545 and parameters: {'criterion': 'squared_error', 'max_features': 'log2', 'max_depth': 9, 'min_samples_split': 4, 'min_samples_leaf': 10}. Best is trial 2 with value: 9.579209481161326.
[I 2025-02-16 14:51:49,304] Trial 4 finished with value: 9.74204750480965 and parameters: {'criterion': 'squared_error', 'max_features': 'log2', 'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 6}. Best is trial 2 w

Fold: 5/5
Model name: random_forest
MAE: 7.5370382439487775
MSE: 90.01348170149315
RMSE: 9.487543501955242
PCC: 0.6857639716580353
Spearman R: 0.6925633047122285
R2 Score: 0.44913089876289547

{'random_forest': {'mae': 7.618735728810615, 'mse': 92.02054531276293, 'rmse': 9.591544899551142, 'pcc': 0.6773153570393651, 'spearman_r': 0.6825826810869848, 'r2_score': 0.4365591564828451}}
